In [ ]:
# !pip install opencv-python
# !conda install -c conda-forge opencv

## 가상화
# conda create -n cv
# conda activate cv
# conda install -c conda-forge jupyterlab glob2 opencv pytest-shutil tensorflow=2.1.0
# pip install tensorflow-addons

In [13]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np

In [14]:
PATH_DEFECT = 'data\\Defect_images\\'
PATH_MASK = 'data\\Mask_images\\'
PATH_NODEFECT = 'data\\NODefect_images\\'

### 데이터셑 불러오기

In [15]:
random.seed(0)

defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '**/*.png')

In [16]:
# Match defect-mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
    num = defect.split('/')[-1].split('_')[0]
    for mask in mask_list:
        num_mask = mask.split('/')[-1].split('_')[0]
        if num == num_mask:
            new_defect_list.append(defect)
            new_mask_list.append(mask)
            break
defect_list = new_defect_list
mask_list = new_mask_list

### 첫번째 데이터셑 구성 (첫번째 발송 데이터)

In [12]:
if os.path.exists('data/dataset/1') is False:
    os.mkdir('data/dataset/1')
for file_name in pass_list + defect_list:
    if random.randint(0, 9) < 2:
        barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        shutil.copy2(file_name, 'data/dataset/1/' + barcode + '.png')

### 두번째 데이터셑 구성

In [18]:
if os.path.exists('data/dataset/2') is False:
    os.mkdir('data/dataset/2')
if os.path.exists('data/dataset/2/OK') is False:
    os.mkdir('data/dataset/2/OK')
if os.path.exists('data/dataset/2/FAIL') is False:
    os.mkdir('data/dataset/2/FAIL')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height
    
    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 2:
            patch = img[:, w:w+height, :]
            cv2.imwrite('data/dataset/2/OK/%04d.png' % idx, patch)
            idx += 1
patch_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_list.append(patch)
random.shuffle(patch_list)
patch_listfraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_listfraction):
    cv2.imwrite('data/dataset/2/FAIL/%04d.png' % idx, patch)

### 세번째 데이터셑 구성

In [19]:
if os.path.exists('data/dataset/3') is False:
    os.mkdir('data/dataset/3')
if os.path.exists('data/dataset/3/OK') is False:
    os.mkdir('data/dataset/3/OK')
if os.path.exists('data/dataset/3/FAIL') is False:
    os.mkdir('data/dataset/3/FAIL')
if os.path.exists('data/dataset/3/MASK') is False:
    os.mkdir('data/dataset/3/MASK')

In [20]:
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2
    
    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 3:
            patch = img[:, w:w+height, :]
            cv2.imwrite('data/dataset/3/OK/%04d.png' % idx, patch)
            idx += 1

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))
random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('data/dataset/3/FAIL/%04d.png' % idx, patch)
    cv2.imwrite('data/dataset/3/MASK/%04d.png' % idx, patch_d)

### 실전 데이터셑 구성